In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5252220704687126302
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3576624695550359711
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Lung\ cancer

/content/drive/.shortcut-targets-by-id/1umanQg67lEFiZ25I1hFeBkjrnHgUDaxL/Lung cancer


In [ ]:
ls

 Data/   'Project for DeepLearning.ipynb'   Tools/
 Model/   results/                          weights.hdf5


In [ ]:
root_path = "Data/"

In [ ]:
import os
# from Model import DenseUnet

import matplotlib.pyplot as plt
import os
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, SpatialDropout2D
from keras.layers import Input, concatenate, UpSampling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.preprocessing.image import load_img
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import random
import cv2

In [ ]:
img = 'Images'
mask = 'masks'

In [ ]:
img_height = 512
img_width = 512
batch_size = 4
img_size = (img_height, img_width)
# save_path = results_path

### Read Images and Masks by Generator

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255,
    rotation_range=30,
    fill_mode='nearest',
    width_shift_range=0.2, 
    height_shift_range=0.2,
    zoom_range=0.3)
    
train_image_generator = datagen.flow_from_directory(
    root_path,
    classes = [img],
    class_mode = None,
    color_mode = 'rgb',
    target_size = img_size,
    batch_size = batch_size,
    seed = 1
)

train_mask_generator = datagen.flow_from_directory(
    root_path,
    classes = [mask],
    class_mode = None,
    color_mode = 'grayscale',
    target_size = img_size,
    batch_size = batch_size,
    seed = 1
)

Found 356 images belonging to 1 classes.
Found 356 images belonging to 1 classes.


In [ ]:
test_path = '../Data/BigNodules/test/'
datagen = ImageDataGenerator(rescale=1. / 255)
    #rotation_range=30,
    #fill_mode='nearest',
    #width_shift_range=0.2, 
    #height_shift_range=0.2,
    #zoom_range=0.3)

test_image_generator = datagen.flow_from_directory(
    test_path,
    classes = ['img'],
    class_mode = None,
    color_mode = 'rgb',
    target_size = img_size,
    batch_size = batch_size,
    seed = 1
)

test_mask_generator = datagen.flow_from_directory(
    test_path,
    classes = ['mask'],
    class_mode = None,
    color_mode = 'grayscale',
    target_size = img_size,
    batch_size = batch_size,
    seed = 1
)

Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


In [ ]:
def train_generator():
  train_generator = zip(train_image_generator, train_mask_generator)
  for (img,mask) in train_generator:
      yield (img,mask)

def test_generator():
  test_generator = zip(test_image_generator, test_mask_generator)
  for (img,mask) in test_generator:
      yield (img,mask)


In [ ]:
train_generator = train_generator()

In [ ]:
test_generator = test_generator()

### Train Model

In [ ]:
from keras.models import Model
from tensorflow.keras.optimizers import Adam
# from model.utils import input_tensor, single_conv, double_conv, deconv, pooling, merge, callback

In [ ]:
# from model.unet import UNet

In [ ]:
smooth = 1.0
width = 32

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
def jacard(y_true, y_pred):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum ( y_true_f * y_pred_f)
    union = K.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)

    return intersection/union

In [ ]:
def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.75
    smooth = 1
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)
    
def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

In [ ]:
def unet(pretrained_weights = None, input_size = (512, 512, 3)):
    inputs = Input(input_size)
    conv1 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3,  padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization(axis = 1)(conv2)
    conv2 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3,  padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization(axis = 1)(conv3)
    conv3 = Conv2D(128, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization(axis = 1)(conv4)
    conv4 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, 3,  padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = BatchNormalization(axis = 1)(conv5)
    conv5 = Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(conv5)

    up6 = Conv2D(256, 2, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    up6 = concatenate([conv4,up6], axis = 3)
    conv6 = SpatialDropout2D(0.35)(up6)
    conv6 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
    conv6 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(128, 2, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    up7 = concatenate([conv3,up7], axis = 3)
    conv7 = SpatialDropout2D(0.35)(up7)
    conv7 = Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(conv7)
    conv7 = Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(64, 2, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    up8 = concatenate([conv2,up8], axis = 3)  
    conv8 = SpatialDropout2D(0.35)(up8)
    conv8 = Conv2D(64, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv8 = Conv2D(64, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(32, 2, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    up9 = concatenate([conv1,up9], axis = 3)
    conv9 = SpatialDropout2D(0.35)(up9)
    conv9 = Conv2D(32, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(32, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3,  padding = 'same', kernel_initializer = 'he_normal')(conv9)

    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer=Adam(learning_rate=1e-6), loss=dice_coef_loss, metrics=dice_coef)
    
    #model.summary()

    # if(pretrained_weights):
    # 	model.load_weights(pretrained_weights)

    return model



In [ ]:
import os

from glob import glob
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.optimizers import *
from keras.models import *
from keras.layers import  *
import pandas as pd
from keras.callbacks import ModelCheckpoint
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import random
import cv2
import numpy as np


def conv_block(x, growth_rate, name):

    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1
    x1 = BatchNormalization(axis=bn_axis,
                                   epsilon=1.001e-5,
                                   name=name + '_0_bn')(x)
    x1 = Activation('relu', name=name + '_0_relu')(x1)
    x1 = Conv2D(4 * growth_rate, 1,
                       use_bias=False,
                       name=name + '_1_conv')(x1)
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                   name=name + '_1_bn')(x1)
    x1 = Activation('relu', name=name + '_1_relu')(x1)
    x1 = Conv2D(growth_rate, 3,
                       padding='same',
                       use_bias=False,
                       name=name + '_2_conv')(x1)
    x = Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
    return x


def dense_block(x, blocks, name):

    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x


def DenseBlock(channels,inputs):


    conv1_1 = Conv2D(channels, (1, 1),activation=None, padding='same')(inputs)
    conv1_1 = BatchActivate(conv1_1)
    conv1_2 = Conv2D(channels//4, (3, 3), activation=None, padding='same')(conv1_1)
    conv1_2 = BatchActivate(conv1_2)

    conv2 = concatenate([inputs,conv1_2])
    conv2_1 = Conv2D(channels, (1, 1), activation=None, padding='same')(conv2)
    conv2_1 = BatchActivate(conv2_1)
    conv2_2 = Conv2D(channels // 4, (3, 3), activation=None, padding='same')(conv2_1)
    conv2_2 = BatchActivate(conv2_2)

    conv3 = concatenate([inputs, conv1_2,conv2_2])
    conv3_1 = Conv2D(channels, (1, 1), activation=None, padding='same')(conv3)
    conv3_1 = BatchActivate(conv3_1)
    conv3_2 = Conv2D(channels // 4, (3, 3), activation=None, padding='same')(conv3_1)
    conv3_2 = BatchActivate(conv3_2)

    conv4 = concatenate([inputs, conv1_2, conv2_2,conv3_2])
    conv4_1 = Conv2D(channels, (1, 1), activation=None, padding='same')(conv4)
    conv4_1 = BatchActivate(conv4_1)
    conv4_2 = Conv2D(channels // 4, (3, 3), activation=None, padding='same')(conv4_1)
    conv4_2 = BatchActivate(conv4_2)
    result = concatenate([inputs,conv1_2, conv2_2,conv3_2,conv4_2])
    return result

def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def spatialAttention(channels, inputs):
    conv5 = Conv2D(channels,3, activation='relu', padding='same', dilation_rate=4, kernel_initializer='he_normal')(inputs)
    conv5 = Dropout(0.5)(conv5)
    channel = BatchActivate(conv5)
    average_pooling = tf.reduce_max(channel, axis=3)
    avg_pool = tf.expand_dims(average_pooling, axis=3)
    max_pooling = tf.reduce_mean(channel, axis=3)
    max_pool = tf.expand_dims(max_pooling, axis=3)
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    cbam_feature = Conv2D(channels, 3, strides=1,padding='same',activation='sigmoid',kernel_initializer='he_normal',use_bias=False)(concat)
    conv5 = multiply([conv5, cbam_feature])
    conv5 = Conv2D(channels,3, activation='relu', padding='same', dilation_rate=2, kernel_initializer='he_normal')(conv5)
    conv5 = Dropout(0.5)(conv5)
    resultt = BatchActivate(conv5)
    return resultt



def DenseUNet(input_size=(512, 512, 3), output = 3, start_neurons=8, block_size=7, lr=1e-3):
    blocks = [1, 2, 4]

    inputs = Input(input_size)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(inputs)
    conv1 = BatchActivate(conv1)
    conv1 = DenseBlock(start_neurons * 1, conv1)
    # pool1 = MaxPooling2D((2, 2))(conv1)
    
    # conv1 = spatialAttention(start_neurons * 1, conv1)
    

    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = DenseBlock(start_neurons * 2, pool1)
    # conv2 = dense_block(pool1, blocks[1], name='conv2')
    # conv2 = spatialAttention(start_neurons * 2, conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = DenseBlock(start_neurons * 4, pool2)
    # conv3 = dense_block(pool2, blocks[2], name='conv3')
    # conv3 = spatialAttention(start_neurons * 4, conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    input_feature1 = Conv2D(start_neurons * 5, (3, 3), activation='relu', padding='same')(pool3)
    
    channel = list(input_feature1.shape)[-1]
    avg_pool = GlobalAveragePooling2D()(input_feature1)
    avg_pool = tf.keras.layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = tf.keras.layers.Dense(channel // 8,
                               activation='relu',
                               kernel_initializer='he_normal',
                               use_bias=True,
                               bias_initializer='zeros')(avg_pool)

    avg_pool = tf.keras.layers.Dense(channel,
                               kernel_initializer='he_normal',
                               use_bias=True,
                               bias_initializer='zeros')(avg_pool)


    max_pool = tf.keras.layers.GlobalMaxPooling2D()(input_feature1)
    max_pool = tf.keras.layers.Reshape((1, 1, channel))(max_pool)
    max_pool = tf.keras.layers.Dense(channel // 8,
                               activation='relu',
                               kernel_initializer='he_normal',
                               use_bias=True,
                               bias_initializer='zeros')(max_pool)

    max_pool = tf.keras.layers.Dense(channel,
                               kernel_initializer='he_normal',
                               use_bias=True,
                               bias_initializer='zeros')(max_pool)

    feature = tf.keras.layers.Add()([avg_pool, max_pool])
    feature = tf.keras.layers.Activation('sigmoid')(feature)

    conv5 = multiply([input_feature1, feature])
    conv5 = Conv2D(start_neurons*5, 3, activation='relu', padding='same', dilation_rate=2, kernel_initializer='he_normal')(conv5)
    conv5 = Dropout(0.5)(conv5)
    convm = BatchActivate(conv5)
    convm = spatialAttention(start_neurons*4, convm)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = BatchActivate(uconv3)
    uconv3 = DenseBlock(start_neurons * 4, uconv3)
    # uconv3 = dense_block(uconv3, blocks[2], name='uconv3')
    uconv3 = spatialAttention(start_neurons*4, uconv3)


    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = BatchActivate(uconv2)
    uconv2 = DenseBlock(start_neurons * 2, uconv2)
    # uconv2 = dense_block(uconv2, blocks[1], name='uconv2')
    uconv2 = spatialAttention(start_neurons*2, uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = BatchActivate(uconv1)
    uconv1 = DenseBlock(start_neurons * 1, uconv1)
    # uconv1 = dense_block(uconv1, blocks[0], name='uconv1')
    uconv1 = spatialAttention(start_neurons*1, uconv1)
    
    output_layer_noActi = Conv2D(output, (3, 3), padding="same", activation=None)(uconv1)
    output_layer = Activation('sigmoid')(output_layer_noActi)

    model = Model(inputs, output_layer)

    return model

In [ ]:
model = DenseUNet(start_neurons=16, input_size=(img_height, img_width, 3), output=1)

model.compile(optimizer=Adam(learning_rate=1e-3),  loss=dice_coef_loss, metrics=dice_coef)

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_201 (Conv2D)            (None, 512, 512, 16  448         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_183 (Batch  (None, 512, 512, 16  64         ['conv2d_201[0][0]']             
 Normalization)                 )                                                           

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Activation,
                                     Dropout, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D,
                                     multiply, Dense, Reshape, Add)
from tensorflow.keras.models import Model

In [ ]:
tf.keras.utils.plot_model(model, to_file='DenseUNet.png', show_shapes=True, show_layer_names=True, expand_nested=True)


In [ ]:
checkpoint = ModelCheckpoint('weights.hdf5', monitor='loss', verbose=1, save_best_only=True)

In [ ]:
# model.fit(train_generator, epochs=50, steps_per_epoch=256, verbose=1, callbacks=[checkpoint], shuffle=True)
# model.save("big_nodule_model.h5")


## Retrain

In [ ]:
filepath="weights.hdf5"
# model.load_weights(filepath)
model.fit(train_generator, batch_size = 8, epochs=100, steps_per_epoch=256, verbose=1, callbacks=[checkpoint], shuffle=True)


Epoch 1/100
256/256 [==============================] - ETA: 0s - loss: 0.9430 - dice_coef: 0.0570
Epoch 1: loss improved from inf to 0.94297, saving model to weights.hdf5
256/256 [==============================] - 181s 643ms/step - loss: 0.9430 - dice_coef: 0.0570
Epoch 2/100
256/256 [==============================] - ETA: 0s - loss: 0.8259 - dice_coef: 0.1741
Epoch 2: loss improved from 0.94297 to 0.82594, saving model to weights.hdf5
256/256 [==============================] - 164s 640ms/step - loss: 0.8259 - dice_coef: 0.1741
Epoch 3/100
256/256 [==============================] - ETA: 0s - loss: 0.7915 - dice_coef: 0.2085
Epoch 3: loss improved from 0.82594 to 0.79147, saving model to weights.hdf5
256/256 [==============================] - 164s 640ms/step - loss: 0.7915 - dice_coef: 0.2085
Epoch 4/100
256/256 [==============================] - ETA: 0s - loss: 0.7475 - dice_coef: 0.2525
Epoch 4: loss improved from 0.79147 to 0.74749, saving model to weights.hdf5
256/256 [=============

In [ ]:
import keras.losses
keras.losses.custom_loss = dice_coef_loss

In [ ]:
model = keras.models.load_model('weights.hdf5', compile=False)

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-3),  loss=dice_coef_loss, metrics=dice_coef)

In [ ]:
model.summary()

In [ ]:
model.fit(train_generator, batch_size = 8, epochs=100, steps_per_epoch=256, verbose=1, callbacks=[checkpoint], shuffle=True)

Epoch 1/100
256/256 [==============================] - ETA: 0s - loss: 0.3478 - dice_coef: 0.6522
Epoch 1: loss improved from inf to 0.34776, saving model to weights.hdf5
256/256 [==============================] - 255s 910ms/step - loss: 0.3478 - dice_coef: 0.6522
Epoch 2/100
256/256 [==============================] - ETA: 0s - loss: 0.3474 - dice_coef: 0.6526
Epoch 2: loss improved from 0.34776 to 0.34740, saving model to weights.hdf5
256/256 [==============================] - 169s 660ms/step - loss: 0.3474 - dice_coef: 0.6526
Epoch 3/100
256/256 [==============================] - ETA: 0s - loss: 0.3483 - dice_coef: 0.6517
Epoch 3: loss did not improve from 0.34740
256/256 [==============================] - 168s 656ms/step - loss: 0.3483 - dice_coef: 0.6517
Epoch 4/100
256/256 [==============================] - ETA: 0s - loss: 0.3437 - dice_coef: 0.6563
Epoch 4: loss improved from 0.34740 to 0.34371, saving model to weights.hdf5
256/256 [==============================] - 169s 662ms/st

### Predict

In [ ]:
from keras.preprocessing import image
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt


In [ ]:
result = model.predict_generator(test_generator, verbose = 1, steps=1)


In [ ]:
r = result[0]
r_img = ((r - r.min()) * (1/(r.max() - r.min()) * 255)).astype('uint8')
r_img = r_img.reshape(512,512)
print(r_img)
(thresh, blackAndWhiteImage) = cv2.threshold(r_img, 100, 1, cv2.THRESH_BINARY)
plt.imshow(r_img, cmap='gray')
plt.imshow(np.squeeze(r_img) , cmap='gray')

In [ ]:
r = result[0]

img = np.squeeze(r)
img = cv2.normalize(img, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8UC1)
print(img)
(thresh, blackAndWhiteImage) = cv2.threshold(img, 100, 1, cv2.THRESH_BINARY)
plt.imshow(img, cmap = 'gray')
  